In [1]:
import sys
sys.path.insert(0, 'caffe/python')
import caffe
import numpy as np
import scipy.misc
import cv2
import scipy.io
import os, sys, argparse
from os.path import join, splitext, split, isfile
import pylayer

In [2]:
caffe.set_mode_gpu()

def convert(data):
    assert data.ndim == 3
    data -= np.array((104.00698793,116.66876762,122.67891434))
    data = data.transpose((2, 0, 1))
    return data

def inverse_convert(data):
    assert data.ndim == 3
    data = data.transpose((1, 2, 0))
    data += np.array((104.00698793,116.66876762,122.67891434))
    return data

def rescale_image(image):
    return image / 255.
    
def convert_label(label):
    mask = label >= 25
    label[mask] = 1
    label[~mask] = 0
    return label

model = 'model/hed_pretrained_bsds.caffemodel'


test_dir = '../examples/imagenet/resized/val' # test images directory




In [3]:
def get_image(image_name):
    
    img = cv2.cvtColor(cv2.imread(image_name), cv2.COLOR_BGR2RGB).astype(np.float32)
    return convert(img)
def get_label(label_name):
    img = cv2.imread(label_name).astype(np.float32)
    return convert_label(np.squeeze(img[:,:,:1]))

diversity_prob = 0.0
epsilon = 16
step = 2
mom = 0.5
smoothing = 0


In [ ]:

def forward_test(dat):
    net.blobs['data'].reshape(1, *dat.shape)
    net.blobs['data'].data[...] = dat
    net.forward()
    return np.array(net.blobs['sigmoid_fuse'].data)

argnet = 'model/hed_test.pt'

net = caffe.Net(argnet, model, caffe.TEST)



In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import scipy

for folder in os.listdir(test_dir):
    for image in os.listdir(os.path.join(test_dir, folder)):
        pixels = get_image(os.path.join(test_dir, folder, image))
        try:
            os.makedirs(os.path.join('imagenet-hed-outputs', folder))
        except:
            pass
        scipy.misc.imsave(os.path.join('imagenet-hed-outputs', folder, image), np.squeeze(forward_test(pixels)))

In [4]:

argnet = 'model/hed_train_attack.pt'


net = caffe.Net(argnet, model, caffe.TRAIN)


In [ ]:
from hed_attack import attack

targ = False
suppress = False
attacks_dir = './imagenet_attacked_{:0.1f}_{:0.1f}_{:0.1f}_{:0.1f}_{}_{}'.format(diversity_prob, epsilon, step, mom, targ, suppress)
try:
    os.makedirs(attacks_dir)
except OSError as e:
    pass
real_dir = './output'

i = 0
for folder in os.listdir(test_dir):
    for image in os.listdir(os.path.join(test_dir, folder)):
        gt = get_label(os.path.join('imagenet-hed-outputs', folder, image))
        pixels = get_image(os.path.join(test_dir, folder, image))
        if suppress:
            att = attack(net, pixels, np.zeros_like(gt), step, epsilon, mom, 10, smoothing, True)
        else:
            att = attack(net, pixels, gt, step, epsilon, mom, 10, smoothing, targ)
        try:
            os.makedirs(join(attacks_dir, folder))
        except:
            pass
        scipy.misc.imsave(join(attacks_dir, folder, image), inverse_convert(att))
        i+=1
    print(i)

/home/christian/.conda/envs/caf/lib/python2.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000
6050
6100
6150
6200
6250
6300
6350
6400
6450
6500
6550
6600
6650
6700
6750
6800
6850
6900
6950
7000
7050
7100
7150
7200
7250
7300
7350
7400
7450
7500
7550
7600
7650
7700
7750
7800
7850
7900
7950
8000
8050
8100
8150
8200
8250
8300
8350
8400
8450
8500
8550
8600
8650
8700
8750
8800
8850
8900
8950
9000
9050
9100
9150
9200
9250
9300
9350
9400
9450
9500
9550
9600
9650
9700
9750
9800
9850
9900
9950
10000
10050
10100
10150
1

Targeted attack transferability experiment

In [ ]:

with open('./map_clsloc.txt') as f:
    dir_to_class = dict([(x[0], x[1]) for x in (y.split() for y in f.read().splitlines())])


In [ ]:
import random
targ = True
attacks_dir = './imagenet_attacked_{:0.1f}_{:0.1f}_{:0.1f}_{:0.1f}_{}'.format(diversity_prob, epsilon, step, mom, targ)
try:
    os.makedirs(attacks_dir)
except OSError as e:
    pass
real_dir = './output'

i = 0
for folder in os.listdir(test_dir):
    for image in os.listdir(os.path.join(test_dir, folder)):
        # get a random edge output
        c = random.choice(os.listdir('imagenet-hed-outputs'))
        im = random.choice(os.listdir(os.path.join('imagenet-hed-outputs', c)))
        
        gt = get_label(os.path.join('imagenet-hed-outputs', folder, image))
        pixels = get_image(os.path.join(test_dir, folder, image))
        att = attack(pixels, gt, targ)
        try:
            os.makedirs(join(attacks_dir, folder))
        except:
            pass
        scipy.misc.imsave(join(attacks_dir, folder, dir_to_class[c] + '_' + image), att.transpose(1, 2, 0))
        i+=1
    print(i)

In [ ]:
from shutil import copyfile

new_dir = './imagenet_shuffled_{:0.1f}_{:0.1f}_{:0.1f}_{:0.1f}_{}_{}'.format(diversity_prob, epsilon, step, mom, targ, smoothing)
try:
    os.makedirs(attacks_dir)
except OSError as e:
    pass

for folder in os.listdir(attacks_dir):
    for image in os.listdir(os.path.join(attacks_dir, folder)):
        # get a random edge output
        new_folder = "%04d" % int(image.split('.')[0].split('_')[0])
        
        try:
            os.makedirs(os.path.join(new_dir, new_folder))
        except OSError as e:
            pass
        copyfile(os.path.join(attacks_dir,folder,image), os.path.join(new_dir, new_folder, image))
    print(i)

In [ ]:
sorted([int(x[1]) for x in dir_to_class.iteritems()])